<a href="https://colab.research.google.com/github/Madelinelai/Deeplearning/blob/main/transferlearning(VGG16).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#嘗試實作遷移學習_VGG16（使用GPU並掛接雲端硬體）
#創建貓狗分類器

In [ ]:
import zipfile
fn = "/content/drive/MyDrive/additional/train.zip"
zf = zipfile.ZipFile(fn)
zf.extractall()

In [ ]:
import glob
import matplotlib.pyplot as plt
from PIL import Image
fns = glob.glob("train/*")
print(len(fns))
img = Image.open(fns[0])
print(img.size)
plt.imshow(img)

In [ ]:
import pandas as pd
dogf =  glob.glob("train/dog.*")
catf =  glob.glob("train/cat.*")
data = pd.DataFrame({
  "path":dogf + catf,
  "target":[0] * len(dogf) + [1] * len(catf)
})
data

In [ ]:
#Use Model的寫法一
from tensorflow.keras.applications.vgg16 import VGG16
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
vgg.output

In [ ]:
#讓後面的分類器小一點，因為ｄａｔａ量沒有很多的寫法二-1 USE:GlobalAveragePool2D
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
t1 = GlobalAveragePooling2D()(vgg.output)
out = Dense(2, activation="softmax")(t1)
model = Model(inputs=vgg.input, outputs=out)
model.summary()


In [ ]:
#承上，用另外一個寫法layer的寫法－sequential，特性一層疊一層 寫法二-2
#補充注意，如果是平行層與平行層的就不行用的sequential寫法喔！
vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
layers = [
    GlobalAveragePooling2D(),
    Dense(2, activation="softmax")
]
model = Sequential(vgg.layers + layers)
model.summary()

#summary的結果
#params　1026 (512*2+2=1026)
#total params 14715714

In [ ]:
#承上，所以我要把前面固定，對trainable＝False做設定，執行結果params會只剩1026
#Total params: 14,715,714
#Trainable params: 1,026
#Non-trainable params: 14,714,688

vgg = VGG16(include_top=False, input_shape=(224, 224, 3))
# trainable設定一定要在compile之前
for l in vgg.layers:
    l.trainable = False
layers = [
    GlobalAveragePooling2D(),
    Dense(2, activation="softmax")
]
model = Sequential(vgg.layers + layers)
model.summary()

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
model.compile(loss=SparseCategoricalCrossentropy(),
              optimizer=Adam(),
              metrics=["accuracy"])